In [1]:
import pandas as pd
import numpy as np
import nltk
import os
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.feature_selection import SelectKBest, chi2


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aom2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
for f in os.listdir("content"):
    print(f)

BinMLResultsada_tfidf_binmat.png
BinMLResultsgb_tfidf_binmat.png
BinMLResultsknn_tfidf_binmat.png
BinMLResultslr_tfidf_binmat.png
BinMLResultsnb_tfidf_binmat.png
BinMLResultsrf_tfidf_binmat.png
BinMLResultssvm_countvec_binmat.png
BinMLResultssvm_tfidf_binmat.png
final_testset.xlsx
final_trainset.xlsx


In [3]:
df_bin = pd.read_excel("content/final_trainset.xlsx")
df_bin['binary2_label'].value_counts()
df_test = pd.read_excel("content/final_testset.xlsx")
df_bin['ABSTRACT'] = df_bin['ABSTRACT'].apply(str.lower)
df_test['ABSTRACT'] = df_test['ABSTRACT'].apply(str.lower)

In [4]:
list(df_test['ABSTRACT'])

['all descriptions of the lambert-eaton syndrome are based on symptomatic patients. we incidentally encountered electromyographic evidence of this syndrome in a patient being studied for another reason. the patient had controlled hypothyroidism and smoked for 9 pack-years but never had symptoms of weakness or had evidence of cancer. detailed electrophysiological and serological evaluations were undertaken in this study. neuromuscular transmission tests were consistent with a presynaptic disorder. autoantibodies specific for p/q-type voltage-gated calcium channels (vgcc) were detected in the serum and persisted throughout 7 years of follow-up. multiple igg markers of thyrogastric autoimmunity also were detected. this demonstration of asymptomatic lambert-eaton syndrome persisting for 8 years provided novel insight into the natural history of the disorder. it is plausible that patients with a subclinical syndrome would have abrupt neuromuscular decompensation when challenged with medicat

In [5]:
# def bin_model(model_type, vectortype,C=1):
#     #vectorization
#     if vectortype == "tfidf":
#         vectorizer = TfidfVectorizer()
#     elif vectortype == "countvec":
#         vectorizer = CountVectorizer()
        
#     bow_train = np.array(vectorizer.fit_transform(list(df_bin['ABSTRACT'])).todense())
#     feature_names = vectorizer.get_feature_names_out()
#     X_train = bow_train
#     y_train = df_bin['binary2_label']
    
#     selector = SelectKBest(score_func=chi2, k=1000)  # Example for feature selection
#     X_train_sel = selector.fit_transform(X_train, y_train)
   

#     #Dimensionality Reduction or Feature Selection


#     if model_type == 'nb':
#         model = MultinomialNB().fit(X_train, y_train)
#     elif model_type == 'lr':
#         model = LogisticRegression(max_iter=1000).fit(X_train, y_train)
#     elif model_type == 'svm':
#         model = LinearSVC(C=C).fit(X_train, y_train)
#     elif model_type == 'knn':
#         model = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
#     elif model_type == 'rf':
#         model = RandomForestClassifier().fit(X_train, y_train)
#     elif model_type == 'ada':
#         model = AdaBoostClassifier(n_estimators=50).fit(X_train, y_train)
#     elif model_type == 'gb':
#         model = GradientBoostingClassifier(n_estimators=50).fit(X_train, y_train)

#     print('Training Accuracy:', model.score(X_train_sel, y_train))

#     bow_test = vectorizer.transform(list(df_test['ABSTRACT']))
#     X_test = bow_test
#     X_test_sel = selector.transform(X_test)  # Same selection applied to test data
    
#     y_test = df_test['binary2_label']
#     y_pred = model.predict(X_test_sel)

#     print('Test Accuracy:', accuracy_score(y_test, y_pred))
#     print('F1 score:', f1_score(y_test, y_pred, average="macro"))
#     print('Precision:', precision_score(y_test, y_pred, average="macro"))
#     print('Recall:', recall_score(y_test, y_pred, average="macro"))

#     fig = plt.figure(figsize=(15,6))
#     ax1 = fig.add_subplot(121)

#     skplt.metrics.plot_confusion_matrix(y_test, y_pred,
#                                     title="Binary Confusion Matrix",
#                                     ax=ax1)
#     plt.savefig('content/BinMLResults' + model_type + '_' + vectortype + '_binmat.png')
#     # return y_test, y_pred
#     return X_train,y_train,feature_names

In [17]:
from sklearn.decomposition import PCA

def bin_model(model_type, vectortype, C=1, n_components=1000):
    # Vectorization
    if vectortype == "tfidf":
        vectorizer = TfidfVectorizer()
    elif vectortype == "countvec":
        vectorizer = CountVectorizer()
        
    X_train = vectorizer.fit_transform(df_bin['ABSTRACT'])
    y_train = df_bin['binary2_label']
    feature_names = vectorizer.get_feature_names_out()
    
    # Dimensionality Reduction with PCA
    pca = PCA(n_components=n_components)
    X_train_reduced = pca.fit_transform(X_train.toarray())  # Ensure dense array if needed

    # Model Training
    if model_type == 'nb':
        print("NB model cannot be used with negative values generated by PCA.")
        return
    elif model_type == 'lr':
        model = LogisticRegression(max_iter=1000)
    elif model_type == 'svm':
        model = LinearSVC(C=C)
    elif model_type == 'knn':
        model = KNeighborsClassifier(n_neighbors=7)
    elif model_type == 'rf':
        model = RandomForestClassifier()
    elif model_type == 'ada':
        model = AdaBoostClassifier(n_estimators=50)
    elif model_type == 'gb':
        model = GradientBoostingClassifier(n_estimators=50)

    model.fit(X_train_reduced, y_train)
    print('Training Accuracy:', model.score(X_train_reduced, y_train))

    # Transform the test set
    X_test = vectorizer.transform(df_test['ABSTRACT'])
    X_test_reduced = pca.transform(X_test.toarray())
    y_test = df_test['binary2_label']
    y_pred = model.predict(X_test_reduced)

    # Evaluation
    print('Test Accuracy:', accuracy_score(y_test, y_pred))
    print('F1 score:', f1_score(y_test, y_pred, average="macro"))
    print('Precision:', precision_score(y_test, y_pred, average="macro"))
    print('Recall:', recall_score(y_test, y_pred, average="macro"))

    return X_train,X_train_reduced, y_train


In [18]:
X_train,X_train_reduced, y_train=bin_model('svm', 'tfidf',C=0.05)#c=0.05

C:\Users\aom2\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training Accuracy: 0.7827568795758647
Test Accuracy: 0.7620192307692307
F1 score: 0.7483363581367211
Precision: 0.7650611699393441
Recall: 0.7437888565822754


In [19]:
X_train,X_train_reduced, y_train=bin_model('svm', 'tfidf',C=0.1)#c=0.1

C:\Users\aom2\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training Accuracy: 0.7992931078010603
Test Accuracy: 0.78125
F1 score: 0.7707572592121597
Precision: 0.7824310776942356
Recall: 0.7663877266387726


In [20]:
# when feature selection = 8000, C=1
# Training Accuracy: 0.9323403181014895
# Test Accuracy: 0.8004807692307693
# F1 score: 0.7909104671935083
# Precision: 0.8032832080200502
# Recall: 0.7860553625038413

# when feature selection = 8200, C=1
# Training Accuracy: 0.9336026255995961
# Test Accuracy: 0.8004807692307693
# F1 score: 0.7904536489151874
# Precision: 0.8043061718434852
# Recall: 0.7853225539559843

In [21]:
X_train,X_train_reduced, y_train=bin_model('svm', 'tfidf',C=1)

C:\Users\aom2\AppData\Local\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Training Accuracy: 0.8226457965160313
Test Accuracy: 0.7836538461538461
F1 score: 0.776183644189383
Precision: 0.7806687844031135
Recall: 0.7736094366829775


In [22]:
 X_train.shape

(7922, 39504)

In [23]:
 X_train_reduced.shape

(7922, 1000)

In [24]:
X_train_reduced

array([[ 0.12219787,  0.01697213,  0.0420014 , ...,  0.00657354,
         0.00307302,  0.00015528],
       [ 0.02598663,  0.02201446,  0.00461343, ...,  0.00925405,
        -0.00760952, -0.01009363],
       [ 0.08053427,  0.13851041, -0.01302804, ..., -0.00386296,
        -0.00855793, -0.02477481],
       ...,
       [-0.00359626, -0.03799162,  0.00905005, ..., -0.00654286,
        -0.00281804,  0.00186728],
       [-0.0351844 ,  0.00195559, -0.00884116, ...,  0.02047284,
         0.00491759, -0.00724833],
       [ 0.01359462, -0.00347871, -0.00996569, ...,  0.01006569,
         0.01343355, -0.00829938]])

In [25]:
X_train,X_train_reduced, y_train=bin_model('lr', 'tfidf')

Training Accuracy: 0.7982832618025751
Test Accuracy: 0.7692307692307693
F1 score: 0.7584204413472706
Precision: 0.7690116206422593
Recall: 0.7544618584970333


In [26]:
X_train,X_train_reduced, y_train=bin_model('ada', 'tfidf')

C:\Users\aom2\AppData\Local\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training Accuracy: 0.7736682655894976
Test Accuracy: 0.7548076923076923
F1 score: 0.7488636363636363
Precision: 0.7492590397154713
Recall: 0.7485048341725173


In [27]:
X_train,X_train_reduced, y_train=bin_model('gb', 'tfidf')

Training Accuracy: 0.7890684170663974
Test Accuracy: 0.7259615384615384
F1 score: 0.7148834860399682
Precision: 0.7209748419939502
Recall: 0.7124081034442002
